In [1]:
# Imports 
import os
import sys
sys.path.append("..")
os.environ["pyna_language"] = 'python'
import pdb
import numpy as np
import openmdao.api as om
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
%matplotlib inline
from pyNA.pyna import pyna

from scipy.interpolate import RegularGridInterpolator

from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Inputs
ac_name = 'a10'

pyna_settings = pyna.load_settings(case_name =ac_name)
pyna_settings.ac_name = ac_name
pyna_settings.theta_flaps = 20.
pyna_settings.theta_slats = -14
pyna_settings.engine_file_name = 'engine_deck_'+ac_name+'.csv'

# Start pyna
py = pyna(settings=pyna_settings)
py.ac.load_aerodynamics(py.settings)
py.engine.load_deck(py.settings)

gamma_approach = -3   # Glide slope [deg]
k_approach = 1.3    # Approach speed ratio (Vapproach/Vstall) [-]

In [8]:
# Compute approach path
c_0 = 346.16136
T_0 = 296.2191226
p_0 = 97834.26355
rho_0 = 1.150789769
mu_0 = 1.83E-05
I_0 = 3.97E+02

f_clmax = RegularGridInterpolator((py.ac.aero['theta_flaps'], py.ac.aero['theta_slats']), py.ac.aero['c_l_max'][0,:,:])
c_l_max = f_clmax((pyna_settings.theta_flaps, pyna_settings.theta_slats))

v_stall = np.sqrt(2*9.80665 * py.ac.mtow / rho_0 / py.ac.af_S_w / c_l_max)
v_approach = k_approach * v_stall
L = 9.80665 * py.ac.mtow * np.cos(gamma_approach * np.pi / 180.)
c_l = L / (0.5* rho_0 * v_approach ** 2 * py.ac.af_S_w)

c_l_interp = RegularGridInterpolator((py.ac.aero['alpha'], py.ac.aero['theta_flaps'], py.ac.aero['theta_slats']), py.ac.aero['c_l'])        
c_l_data = c_l_interp((py.ac.aero['alpha'], py.settings.theta_flaps, py.settings.theta_slats))
c_d_interp = RegularGridInterpolator((py.ac.aero['alpha'], py.ac.aero['theta_flaps'], py.ac.aero['theta_slats']), py.ac.aero['c_d'])
c_d_data = c_d_interp((py.ac.aero['alpha'], py.settings.theta_flaps, py.settings.theta_slats))     

alpha = np.interp(c_l, c_l_data, py.ac.aero['alpha'])
c_d = np.interp(alpha, py.ac.aero['alpha'], c_d_data)

F_req = (c_d * 0.5 * rho_0 * v_approach**2 * py.ac.af_S_w) + py.ac.mtow*9.80065*np.sin(gamma_approach * np.pi / 180.)

# Create engine deck interpolant
F_n_interp = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['F_n'])

# Compute thrust available
F_avail = F_n_interp((0., v_approach/c_0, 1.))

# Compute thrust setting
TS = F_req / (py.ac.n_eng * F_avail)

# Compute time to fly trajectory
x_init = - 5815.8888
z_init = np.abs(x_init)*np.tan(-gamma_approach*np.pi/180)
s = np.sqrt(x_init**2 + z_init**2)
t_traj = s / v_approach

In [9]:
# Create trajectory data frame 
t = pd.DataFrame()

t['t_source [s]'] = np.arange(0, t_traj, 0.5)
t['X [m]'] = x_init+v_approach*t['t_source [s]']*np.cos(gamma_approach*np.pi/180)
t['Y [m]'] = np.zeros(np.size(t['t_source [s]']))
t['Z [m]'] = z_init+v_approach*t['t_source [s]']*np.sin(gamma_approach*np.pi/180)
t['V [m/s]'] = v_approach*np.ones(np.size(t['t_source [s]']))
t['M_0 [-]'] = (v_approach/c_0)*np.ones(np.size(t['t_source [s]']))
t['F_n [N]'] = F_avail*TS*np.ones(np.size(t['t_source [s]']))
t['TS [-]'] = TS*np.ones(np.size(t['t_source [s]']))
t['c_0 [m/s]'] = c_0*np.ones(np.size(t['t_source [s]']))
t['T_0 [K]'] = T_0*np.ones(np.size(t['t_source [s]']))
t['p_0 [Pa]'] = p_0*np.ones(np.size(t['t_source [s]']))
t['rho_0 [kg/m3]'] = rho_0*np.ones(np.size(t['t_source [s]']))
t['mu_0 [kg/ms]'] = mu_0*np.ones(np.size(t['t_source [s]']))
t['I_0 [kg/m2s]'] = I_0*np.ones(np.size(t['t_source [s]']))
t['alpha [deg]'] = alpha*np.ones(np.size(t['t_source [s]']))
t['gamma [deg]'] = gamma_approach*np.ones(np.size(t['t_source [s]']))
t['Airframe LG [-]'] = np.ones(np.size(t['t_source [s]']))
t['Airframe delta_f [deg]'] = py.settings.theta_flaps * np.ones(np.size(t['t_source [s]']))
t['Airframe delta_s [deg]'] = py.settings.theta_flaps * np.ones(np.size(t['t_source [s]']))


In [10]:
# Create engine dataframe 
e = pd.DataFrame()

e['t_source [s]'] = np.arange(0, t_traj, 0.5)
e['Ne [-]'] = py.ac.n_eng*np.ones(np.size(t['t_source [s]']))
e['TS [-]'] = TS*np.ones(np.size(t['t_source [s]']))

f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['mdot_i_c'])
e['Core mdot [kg/s]'] = f_int((0, v_approach/c_0, TS))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['Pti_c'])
e['Core Pt [Pa]'] = f_int((0, v_approach/c_0, TS))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['Tti_c'])
e['Core Tti [K]'] = f_int((0, v_approach/c_0, TS))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['Ttj_c'])
e['Core Ttj [K]'] = f_int((0, v_approach/c_0, TS))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['DTt_des_c'])
e['Core DT_t [K]'] = f_int((0, v_approach/c_0, TS))
e['LPT rho_e [kg/m3]'] = np.zeros(np.size(t['t_source [s]']))
e['LPT c_e [m/s]'] = np.zeros(np.size(t['t_source [s]']))
e['HPT rho_i [kg/m3]'] = np.zeros(np.size(t['t_source [s]']))
e['HPT c_i [m/s]'] = np.zeros(np.size(t['t_source [s]']))

f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['A_j'])
e['Jet A [m2]'] = f_int((0, v_approach/c_0, TS))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['rho_j'])
e['Jet rho [kg/m3]'] = f_int((0, v_approach/c_0, TS))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['Tt_j'])
e['Jet Tt [K]'] = f_int((0, v_approach/c_0, TS))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['V_j'])
e['Jet V [m/s]'] = f_int((0, v_approach/c_0, TS))
e['Jet M [-]'] = np.ones(np.size(t['t_source [s]']))
e['Jet delta [deg]'] = np.zeros(np.size(t['t_source [s]']))
e['Airframe LG [-]'] = np.ones(np.size(t['t_source [s]']))
e['Airframe delta_f [deg]'] = py.settings.theta_flaps*np.ones(np.size(t['t_source [s]']))

f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['mdot_f'])
e['Fan mdot in [kg/s]'] = f_int((0, v_approach/c_0, TS))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['N_f'])
e['Fan N [rpm]'] = 2.5*f_int((0, v_approach/c_0, TS))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['DTt_f'])
e['Fan delta T [K]'] = f_int((0, v_approach/c_0, TS))
e['Fan B [-]'] = 25*np.ones(np.size(t['t_source [s]']))
e['Fan V [-]'] = 48*np.ones(np.size(t['t_source [s]']))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['d_f'])
e['Fan d [m]'] = f_int((0, v_approach/c_0, TS))
e['Fan M_d [-]'] = 1.68*np.ones(np.size(t['t_source [s]']))
f_int = RegularGridInterpolator((py.engine.deck['z'], py.engine.deck['M_0'], py.engine.deck['TS']), py.engine.deck['A_f'])
e['Fan A [m2]'] = f_int((0, v_approach/c_0, TS))
e['Fan RSS [%]'] = 300.*np.ones(np.size(t['t_source [s]']))
e['Fan IGV [-]'] = np.zeros(np.size(t['t_source [s]']))
e['Fan ID [-]'] = np.zeros(np.size(t['t_source [s]']))

In [11]:
# Save trajectories
t.to_csv('../cases/'+ac_name+'/trajectory/trajectory_approach.csv', index=False)
e.to_csv('../cases/'+ac_name+'/engine/engine_approach.csv', index=False)